# **cycle detection** 

# Set up the environment
### This command will only need to be executed once

In [1]:
!pip install networkx
!pip install scipy

In [2]:
import numpy as np
import time
import networkx as nx

# Generate graph
You don't need to handle the code here

In [3]:
#@title
def gen_graph(node_num, edge_num):
    graph = list()
    rec = list()
    for _ in range(edge_num):
        while True:
            edge = np.random.randint(0, node_num, 2)
            if edge[0] != edge[1]:
                if len(rec) == 0:
                    rec.append('{} {}'.format(edge[0], edge[1]))
                else:
                    if '{} {}'.format(edge[0], edge[1]) not in rec:
                        graph.append(edge.tolist())
                        rec.append('{} {}'.format(edge[0], edge[1]))

                break
    return graph

def convert_p1(graph_list):
    p1_list = list()
    for graph in graph_list:
        node_num = np.max(graph)+1
        p1_matrix = list()
        for x, y in graph:
            row = [0] * node_num
            row[x] = -1
            row[y] = 1
            p1_matrix.append(row)
        p1_list.append(p1_matrix)
    return p1_list

def convert_p2(graph_list):
    p2_list = list()
    for graph in graph_list:
        node_num = np.max(graph)+1
        p2_matrix = np.zeros((node_num, node_num), dtype=int).tolist()
        for x, y in graph:
            p2_matrix[x][y] = 1
        p2_list.append(p2_matrix)
    return p2_list

### **Modify the number of nodes and edges here!**

In [4]:
def gen_graph_list(size):
    graph_list = list()
    for _ in range(size):
    # ---------------------------------------------------------------------------- #
    #     Modify the number of nodes and edges here           #
    # ---------------------------------------------------------------------------- #
        node_num = np.random.randint(20, 60)
        edge_num = np.random.randint(10, 50)
        graph_list.append(gen_graph(node_num, edge_num))
    return graph_list

### **Change the number of graph you want to generate here!**

In [12]:
def get_p1(seed):
    seed = int(seed[2:])
    np.random.seed(seed*10+1)
    graph_list = gen_graph_list(8)
    return graph_list

def get_p2(the_seed):
    the_seed = int(the_seed[2:])
    np.random.seed(the_seed*10+2)
    graph_list = gen_graph_list(8)
    return graph_list

# **Cycle detection for p1 and p2**

In [49]:
from scipy.sparse import *
import numpy as np

def p1_has_cycle(sets):
    # return True if the graph has cycle; return False if not
    '''
        `print(sets)` to show what the matrix looks like
        If we have a directed graph with 2->3 4->1 3->5 5->2 0->1
              0  1  2  3  4  5
            0  0  0 -1  1  0  0
            1  0  1  0  0 -1  0
            2  0  0  0 -1  0  1
            3  0  0  1  0  0 -1
            4 -1  1  0  0  0  0
        The size of the matrix is (5,6)
        
        Logic Here: add row with 1 to the rows with -1 under Same Column.
        Then append the newly generated rows to the end while deleting 
        the first row.
        Keep updating until termination 1) find row with all 0, return True
        2) the whole matrix has only one row left
    '''
    graph = np.array(sets)
    while graph.shape[0] > 1:
        try:
            col = np.where(graph[0] == 1)[0][0]
            coords = np.where(graph == -1)
            if col in coords[1]:
                rows = np.where(coords[1] == col)[0]
                for row in rows:
                    newrow = graph[0] + graph[row]
                    # this newrow ends up like a circle
                    if sum(newrow) == 0:
                        return True
                    np.vstack(graph, newrow)
            # delete first row
            graph = graph[1:]
            
        except:
            # case no 1 exist in the row/edge, indicating row is all 
            return True
            

    return False

Please finish the function of **p2_has_cycle()**.

In [58]:
from scipy.sparse import *
import numpy as np

def p2_has_cycle(sets):
    # TODO
    # return True if the graph has cycle; return False if not
    '''
      HINT: You can `print(sets)` to show what the matrix looks like
        If we have a directed graph with 2->3 4->1 3->5 5->2 0->1
               0  1  2  3  4  5
            0  0  1  0  0  0  0
            1  0  0  0  0  0  0
            2  0  0  0  1  0  0
            3  0  0  0  0  0  1
            4  0  1  0  0  0  0
            5  0  0  1  0  0  0
        The size of the matrix is (6,6)
    matrix = graph ^ N where N is number of nodes
    if matrix has 1 in diagonal, it has circle
    '''
    graph = csr_matrix(sets)
    num_node = len(sets)
    
    M = csr_matrix(np.identity(num_node))
    for i in range(num_node):
        M = M.dot(graph)
        if sum(M.diagonal()) > 0:
            return True
    return False

### code from judgeURL
```python
def p1_main(seed = "r10"):
    
    p1_list = list()
    p1_list = get_p1(seed)
        
    p1_list_converted = convert_p1(p1_list)
    start_time = time.time()
    for i in range(len(p1_list)):
      
        # if(i!=230):
        #     continue
        graph = nx.DiGraph(p1_list[i])
        has_cycle = True
        try:
            res = nx.find_cycle(graph)
        except:
            has_cycle = False
        if p1_has_cycle(p1_list_converted[i]) != has_cycle:
            print('Bug in the {}th graph. P1.'.format(i))
    print("--- Execution time for p1: %7f seconds ---" % (time.time() - start_time))
    
    
def p2_main(the_seed = "r10"):
    p2_list = list()
    p2_list = get_p2(the_seed)
        
    p2_list_converted = convert_p2(p2_list)
    start_time = time.time()
    for i in range(len(p2_list)):
        graph = nx.DiGraph(p2_list[i])
        has_cycle = True
        try:
            res = nx.find_cycle(graph)
        except:
            has_cycle = False
        if p2_has_cycle(p2_list_converted[i]) != has_cycle:
            print('Bug in the {}th graph. P2.'.format(i))
    print("--- Execution time for p2: %7f seconds ---" % (time.time() - start_time))
```

In [70]:
#@title # Main function of p1 and p2
#@markdown Run me to get testing code...
# # @markdown You should not change the code here.
judgeURL = "https://gist.githubusercontent.com/jeffeuxMartin/13e921442595ac37423b92ab3d51bafe/raw/a33beee8f8180f517c6c05f4a041ef3e73909d10/judge_code__lahw1.py"

import urllib.request
with urllib.request.urlopen(judgeURL) as response:
    exec(response.read())

# **Testing**

In [71]:
#@markdown You can input your student ID as the seed of the random function.
#@markdown Feel free to change it to other student IDs to see whether your function works on different graphs.

seed_id = "009901000" #@param {type: "string"}

In [72]:
p1_main(seed_id)

Bug in the 3th graph. P1.
Bug in the 4th graph. P1.
--- Execution time for p1: 0.009450 seconds ---


In [57]:
p2_main(seed_id)

--- Execution time for p2: 0.038456 seconds ---
